In [38]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import matplotlib.pyplot as plt


Fontconfig warning: ignoring UTF-8: not a valid region tag
Matplotlib is building the font cache; this may take a moment.


In [33]:
# 1. Download data

start_date = '2022-01-01'
end_date = '2023-12-31'

# Download universe data
tickers = ['AAPL', 'NVDA', 'MSFT', 'AMZN', 'TSLA', 'GOOGL', 'META', 'NFLX', 'DIS', 'WMT', 'CVX', 'XOM', 'BP', 'SHEL']
df = pd.DataFrame()

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.droplevel(['Ticker'])
    data = data[['Close']]
    data = data.rename(columns={'Close': ticker})
    df = pd.concat([df, data], axis=1)

for col in df.columns:
    df[col] = df[col].pct_change()
df = df.dropna()


# Download ETF data
ETF_tickers = ['XLSR', 'XLC', 'XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLRE', 'XLK', 'XLU']
df_ETF = pd.DataFrame()

for ticker in ETF_tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.droplevel(['Ticker'])
    data = data[['Close']]
    data = data.rename(columns={'Close': ticker})
    df_ETF = pd.concat([df_ETF, data], axis=1)

for col in df_ETF.columns:
    df_ETF[col] = df_ETF[col].pct_change()
df_ETF = df_ETF.dropna()


# Process PCA data
pca = PCA(n_components=len(df_ETF.columns))
pca.fit(df_ETF)
df_ETF_PCA = pca.transform(df_ETF)
df_ETF_PCA = pd.DataFrame(df_ETF_PCA, columns=[f'PCA_{i}' for i in range(df_ETF_PCA.shape[1])])
df_ETF_PCA.index = df_ETF.index

explained_variance = pca.explained_variance_ratio_
explained_variance_cumsum = np.cumsum(explained_variance)
for i in range(len(explained_variance_cumsum)):
    if explained_variance_cumsum[i] >= 0.95:
        df_ETF_PCA = df_ETF_PCA.drop(columns=[f'PCA_{i}'])


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [41]:
def fit_ou_process_returns(returns, dt=1.0):
    """
    Fit Ornstein-Uhlenbeck parameters to return data using Maximum Likelihood Estimation
    
    Parameters:
    returns: numpy array or pandas series of returns (not prices)
    dt: time step between observations (1.0 for daily returns)
    
    Returns:
    theta: mean reversion rate
    mu: long-term mean return
    sigma: volatility of returns
    """
    # Convert to numpy array if pandas
    if isinstance(returns, pd.Series):
        returns = returns.values
    
    # For returns, we don't need to take differences
    x = returns[:-1]
    y = returns[1:]
    n = len(x)
    
    # Define negative log likelihood function
    def neg_log_likelihood(params):
        theta, mu, sigma = params
        
        # Expected next return according to OU
        expected_y = x * np.exp(-theta * dt) + mu * (1 - np.exp(-theta * dt))
        
        # Variance
        var = (sigma**2 / (2 * theta)) * (1 - np.exp(-2 * theta * dt))
        
        # Log likelihood
        log_likelihood = -0.5 * n * np.log(2 * np.pi * var) - \
                        0.5 * np.sum((y - expected_y)**2) / var
        
        return -log_likelihood
    
    # Initial guess for parameters
    initial_guess = [2.0, np.mean(returns), np.std(returns)]
    
    # Bounds for parameters (all positive for theta and sigma)
    bounds = ((0.0001, None), (None, None), (0.0001, None))
    
    # Optimize
    result = minimize(neg_log_likelihood, initial_guess, bounds=bounds)
    
    theta, mu, sigma = result.x
    
    return float(theta), float(mu), float(sigma)



(3.4348899608718115, 0.0012415076207406995, 0.04755836657029318)

In [32]:
df_ETF_train, df_ETF_test, df_train, df_test = train_test_split(df_ETF, df, test_size=0.2, random_state=42)

regression_models = {}

for ticker in df.columns:
    regression_models[ticker] = LinearRegression()
    regression_models[ticker].fit(df_ETF_train, df_train[ticker])

    intercept = regression_models[ticker].intercept_
    coefficients = regression_models[ticker].coef_
    residuals = df_train[ticker] - regression_models[ticker].predict(df_ETF_train)
    print(f"Residuals for {ticker}: {residuals}")



Residuals for AAPL: Date
2022-12-30    0.002222
2023-09-26   -0.005206
2022-02-01   -0.002417
2023-04-18    0.005411
2023-05-02    0.006424
                ...   
2022-06-07    0.003924
2023-02-01   -0.015122
2023-05-24    0.007153
2023-09-28   -0.001219
2022-06-01    0.001729
Name: AAPL, Length: 400, dtype: float64
Residuals for NVDA: Date
2022-12-30   -0.000977
2023-09-26    0.018599
2022-02-01   -0.003252
2023-04-18    0.013624
2023-05-02   -0.020079
                ...   
2022-06-07   -0.007636
2023-02-01    0.024329
2023-05-24    0.004087
2023-09-28   -0.002737
2022-06-01   -0.024941
Name: NVDA, Length: 400, dtype: float64
Residuals for MSFT: Date
2022-12-30   -0.004834
2023-09-26    0.001989
2022-02-01   -0.003489
2023-04-18   -0.000769
2023-05-02    0.007163
                ...   
2022-06-07    0.003046
2023-02-01   -0.006957
2023-05-24   -0.001440
2023-09-28    0.000507
2022-06-01    0.003366
Name: MSFT, Length: 400, dtype: float64
Residuals for AMZN: Date
2022-12-30    0.00003

In [30]:
df_train

,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5
Date,,,,,,
2023-06-13,0.026231,0.001768,0.001565,0.012112,0.005103,-0.001755
2022-04-20,-0.001979,0.018692,0.037637,0.020388,0.003406,0.015687
2023-07-03,0.011693,-0.000928,0.005074,-0.001842,0.008612,0.004138
2022-08-17,-0.024076,0.013368,0.002743,-0.002638,0.002450,0.008384
2022-06-03,-0.046718,0.027414,0.001923,-0.000746,0.001997,-0.004250
...,...,...,...,...,...,...
2023-05-23,-0.033643,0.014481,-0.006702,-0.008608,0.002278,0.006403
2022-05-09,-0.113395,-0.053004,0.019076,0.014501,-0.017567,-0.005970
2022-04-22,-0.087195,-0.012037,-0.006411,-0.013164,0.014571,0.004611
